In [1]:
import os
import numpy as np
import random 

In [2]:
train_dir = "images/train"
test_dir = "images/test"

In [3]:
img_size = 50

In [4]:
Class = ["cats","dogs"]

In [5]:
os.listdir("images/train/cats")

['cat.6973.jpg',
 'cat.9792.jpg',
 'cat.6611.jpg',
 'cat.10857.jpg',
 'cat.10023.jpg',
 'cat.2450.jpg',
 'cat.6224.jpg',
 'cat.4378.jpg',
 'cat.11790.jpg',
 'cat.9831.jpg',
 'cat.6256.jpg',
 'cat.8399.jpg',
 'cat.11457.jpg',
 'cat.8949.jpg',
 'cat.8051.jpg',
 'cat.11529.jpg',
 'cat.3836.jpg',
 'cat.3592.jpg',
 'cat.11436.jpg',
 'cat.10859.jpg',
 'cat.81.jpg',
 'cat.9860.jpg',
 'cat.1140.jpg',
 'cat.6386.jpg',
 'cat.7630.jpg',
 'cat.3770.jpg',
 'cat.9051.jpg',
 'cat.585.jpg',
 'cat.9181.jpg',
 'cat.6902.jpg',
 'cat.3422.jpg',
 'cat.2544.jpg',
 'cat.10493.jpg',
 'cat.1347.jpg',
 'cat.11320.jpg',
 'cat.4685.jpg',
 'cat.3659.jpg',
 'cat.8678.jpg',
 'cat.6378.jpg',
 'cat.6787.jpg',
 'cat.660.jpg',
 'cat.5156.jpg',
 'cat.2286.jpg',
 'cat.1057.jpg',
 'cat.1346.jpg',
 'cat.8405.jpg',
 'cat.10452.jpg',
 'cat.8960.jpg',
 'cat.2380.jpg',
 'cat.4016.jpg',
 'cat.7616.jpg',
 'cat.3494.jpg',
 'cat.5299.jpg',
 'cat.1327.jpg',
 'cat.9415.jpg',
 'cat.4410.jpg',
 'cat.2207.jpg',
 'cat.9709.jpg',
 'cat.57

In [7]:
!pip install opencv-python

     |████████████████████████████████| 50.4 MB 767 kB/s eta 0:00:01     |█████████▋                      | 15.2 MB 260 kB/s eta 0:02:16     |██████████████▍                 | 22.6 MB 1.7 MB/s eta 0:00:17     |█████████████████████▊          | 34.2 MB 265 kB/s eta 0:01:02     |███████████████████████         | 36.3 MB 619 kB/s eta 0:00:23


In [8]:
import cv2

def create_data(data_dir):
    dataset = []
    for i in Class:
        class_dir = os.path.join(data_dir , i)
        class_num = Class.index(i)
        for img in os.listdir(class_dir):
            try:
                img_array = cv2.imread(os.path.join(class_dir , img))
                img_array = cv2.resize(img_array,(50,50))
                gray_image = cv2.cvtColor(img_array , cv2.COLOR_BGR2GRAY)
                dataset.append([gray_image ,class_num])
            except:
                pass
        random.shuffle(dataset)
    return dataset

In [9]:
train_data = create_data(train_dir)
test_data = create_data(test_dir)

In [10]:
train_data[0]


[array([[209, 214, 201, ...,  29,  25,  19],
        [125, 116,  49, ...,  22,  21,  17],
        [105, 109,  86, ...,  19,  24,  39],
        ...,
        [147, 146, 133, ...,  96,  94, 101],
        [147, 135, 140, ...,  91,  91,  89],
        [ 97,  91,  80, ..., 104, 123,  88]], dtype=uint8),
 1]

In [11]:
len(train_data)

23000

In [12]:
len(test_data)

2000

In [13]:
X_train = []
y_train = []
for feature,label in train_data:
    X_train.append(feature)
    y_train.append(label)

In [14]:
X_test = []
y_test = []
for feature,label in test_data:
    X_test.append(feature)
    y_test.append(label)

In [15]:
X_train = np.array(X_train).reshape(-1,img_size,img_size,1)
y_train = np.array(y_train)
X_train = X_train/255.0

In [16]:
X_test = np.array(X_test).reshape(-1,img_size,img_size,1)
y_test = np.array(y_test)
X_test = X_test/255.0

In [17]:
X_train.shape

(23000, 50, 50, 1)

In [18]:
X_train.shape[1:]

(50, 50, 1)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Conv2D , MaxPooling2D ,Flatten

In [21]:
model = Sequential()

model.add(Conv2D(64 , (3,3) ,input_shape = X_train.shape[1:] ,activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

model.add(Conv2D(128 , (3,3) ,padding = "same", activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

model.add(Conv2D(96 , (3,3) ,padding = "same", activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2) , strides = (2,2)))

model.add(Flatten())

model.add(Dense(activation='relu' , units =128 ))

model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss="binary_crossentropy" , optimizer= 'adam' ,metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 96)        110688    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 96)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3456)              0

In [23]:
model_result = model.fit(X_train,y_train , validation_data = (X_test ,y_test) , epochs =15 ,batch_size=32)

Epoch 1/15
719/719 [==============================] - 167s 230ms/step - loss: 0.6541 - accuracy: 0.5980 - val_loss: 0.5605 - val_accuracy: 0.7070
Epoch 2/15
719/719 [==============================] - 161s 224ms/step - loss: 0.5092 - accuracy: 0.7484 - val_loss: 0.5086 - val_accuracy: 0.7500
Epoch 3/15
719/719 [==============================] - 194s 270ms/step - loss: 0.4333 - accuracy: 0.7990 - val_loss: 0.4013 - val_accuracy: 0.8175
Epoch 4/15
719/719 [==============================] - 168s 234ms/step - loss: 0.3628 - accuracy: 0.8364 - val_loss: 0.4229 - val_accuracy: 0.8135
Epoch 5/15
719/719 [==============================] - 168s 233ms/step - loss: 0.3071 - accuracy: 0.8654 - val_loss: 0.3686 - val_accuracy: 0.8440
Epoch 6/15
719/719 [==============================] - 166s 231ms/step - loss: 0.2652 - accuracy: 0.8865 - val_loss: 0.3752 - val_accuracy: 0.8405
Epoch 7/15
719/719 [==============================] - 167s 232ms/step - loss: 0.2137 - accuracy: 0.9084 - val_loss: 0.3576 -

In [24]:
model.save("cat-dog.h5")

In [30]:
img_arr = cv2.imread("images/train/cats/cat.3730.jpg", cv2.IMREAD_GRAYSCALE)
img_arr

array([[113, 105, 105, ..., 197, 191, 191],
       [113, 106, 106, ..., 196, 191, 191],
       [113, 106, 107, ..., 196, 192, 192],
       ...,
       [161, 156, 160, ..., 168, 164, 164],
       [159, 155, 157, ..., 168, 164, 164],
       [157, 154, 155, ..., 168, 165, 165]], dtype=uint8)

In [31]:
img_arr = cv2.resize(img_arr,(50,50))


In [33]:
img = np.expand_dims(img_arr,axis=0)
img =np.resize(img,(1,50,50,1))
img= img/255.0
y_pred=model.predict(img)
y_pred

array([[6.0397564e-13]], dtype=float32)